In [3]:
import pandas as pd

In [33]:
df = pd.read_csv('heart.csv')

In [34]:
df['target'].unique()

array([0, 1])

In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os
import tensorflow as tf

from tensorflow.keras.layers import TextVectorization

In [48]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [49]:
df.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1
1024,54,1,0,120,188,0,1,113,0,1.4,1,1,3,0


In [38]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [39]:
df['target'].value_counts()

target
1    526
0    499
Name: count, dtype: int64

In [40]:
X = df.drop(columns ='target', axis = 1)
Y=df['target']

In [41]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2


In [42]:
Y

0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: target, Length: 1025, dtype: int64

In [43]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify =Y, random_state = 2)

In [113]:
print(X.shape, X_train.shape, X_test.shape)
from sklearn.preprocessing import StandardScaler

(1025, 13) (820, 13) (205, 13)


In [114]:
scaler = StandardScaler()

In [115]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [116]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled,Y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled,Y_test))

In [117]:
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(buffer_size=len(X_train_scaled))
train_dataset = train_dataset.batch(16)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE) 
test_dataset = test_dataset.batch(16)

In [118]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

In [119]:
model = Sequential()

In [127]:
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu', kernel_regularizer=l2(0.01)))  
model.add(Dropout(0.6))  
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))  
model.add(Dropout(0.6)) 
model.add(Dense(1, activation='sigmoid'))  

In [128]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [129]:
history = model.fit(train_dataset, validation_data=test_dataset, epochs=50)

Epoch 1/50
52/52 [==============================] - 1s 3ms/step - loss: 1.0826 - accuracy: 0.5500 - val_loss: 0.9399 - val_accuracy: 0.8049
Epoch 2/50
52/52 [==============================] - 0s 1ms/step - loss: 0.8841 - accuracy: 0.7390 - val_loss: 0.7822 - val_accuracy: 0.8537
Epoch 3/50
52/52 [==============================] - 0s 1ms/step - loss: 0.7625 - accuracy: 0.8171 - val_loss: 0.6677 - val_accuracy: 0.8829
Epoch 4/50
52/52 [==============================] - 0s 1ms/step - loss: 0.6670 - accuracy: 0.8610 - val_loss: 0.5919 - val_accuracy: 0.8780
Epoch 5/50
52/52 [==============================] - 0s 1ms/step - loss: 0.6281 - accuracy: 0.8439 - val_loss: 0.5503 - val_accuracy: 0.8829
Epoch 6/50
52/52 [==============================] - 0s 1ms/step - loss: 0.5888 - accuracy: 0.8598 - val_loss: 0.5107 - val_accuracy: 0.8829
Epoch 7/50
52/52 [==============================] - 0s 1ms/step - loss: 0.5641 - accuracy: 0.8585 - val_loss: 0.4922 - val_accuracy: 0.8976
Epoch 8/50
52/52 [==

In [130]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

13/13 [==============================] - 0s 1ms/step - loss: 0.3721 - accuracy: 0.9317
Test Accuracy: 93.17%
